In [1]:
from riotwatcher import LolWatcher, ApiError
from riot_api import key
import pandas as pd
from datetime import datetime

region = 'euw1'

lol = LolWatcher(key)

In [2]:
me = lol.summoner.by_name(region,'Katiounette')
puuid = me['puuid']

In [28]:
matchs = lol.match.matchlist_by_puuid(region,puuid, count = 100)
i = 100
while (lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i) != []):
    matchs.extend(lol.match.matchlist_by_puuid(region,puuid, count = 100, start = i))
    i = i + 100

In [29]:
matchData = pd.DataFrame({'playerID' : [],'matchID' : [], 'duration (s)' : [], 'start' : [], 'end' : [], 'championName' : [], 'championId' : [], 'pings' : [], 'ePings' : []})
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings


In [30]:
for j,i in enumerate(matchs):
    p = lol.match.by_id(region,i)
    for player in p['info']['participants']:
        if player['puuid'] == puuid:
            cName = player['championName']
            cId = player['championId']
            ePing = player['enemyMissingPings']
            pings = player["allInPings"] + player["assistMePings"] + player["baitPings"] + player["basicPings"] + player["commandPings"] + player["dangerPings"] + player["getBackPings"] + player["holdPings"] + player["needVisionPings"] + player["onMyWayPings"] + player["pushPings"] + player['visionClearedPings'] + player['enemyMissingPings'] + player['enemyVisionPings']
    matchData.loc[j,:] = [puuid,p['metadata']['matchId'],p['info']['gameDuration'], p['info']['gameStartTimestamp'], p['info']['gameEndTimestamp'], cName, cId, pings, ePing]
    if j == 100:
        break

In [31]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363787945,1707.0,1.681672e+12,1.681674e+12,Kaisa,145.0,11.0,6.0
1,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363740926,915.0,1.681670e+12,1.681671e+12,Tristana,18.0,2.0,2.0
2,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363690986,1416.0,1.681669e+12,1.681670e+12,MissFortune,21.0,7.0,2.0
3,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6360825821,2146.0,1.681504e+12,1.681506e+12,Kaisa,145.0,5.0,3.0
4,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6359215935,1627.0,1.681410e+12,1.681412e+12,MissFortune,21.0,2.0,1.0
...,...,...,...,...,...,...,...,...,...
96,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301057974,2622.0,1.677882e+12,1.677885e+12,MissFortune,21.0,5.0,3.0
97,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300979268,2050.0,1.677879e+12,1.677881e+12,MissFortune,21.0,1.0,1.0
98,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300902911,1992.0,1.677877e+12,1.677879e+12,MissFortune,21.0,5.0,3.0
99,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300315307,1220.0,1.677851e+12,1.677852e+12,MissFortune,21.0,1.0,0.0


In [32]:
def getHourCat(i):
    h = matchData.loc[i,'startHour']
    if h >= 6 and h < 12:
        matchData.loc[i,'startHourCat'] = 'Morning'
    if h >= 12 and h < 17:
        matchData.loc[i,'startHourCat'] = "Afternoon"
    if h >= 17 and h < 21:
        matchData.loc[i,'startHourCat'] = "Evening"
    else :
        matchData.loc[i,'startHourCat'] = "Night"

In [33]:
for i in range(matchData.shape[0]):
    matchData.loc[i,'start'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'start']/1000))
    matchData.loc[i,'end'] = pd.Timestamp(datetime.fromtimestamp(matchData.loc[i,'end']/1000))
    matchData.loc[i,'weekDay'] = matchData.loc[i,'start'].dayofweek
    matchData.loc[i,'weekDayName'] = matchData.loc[i,'start'].day_name()
    matchData.loc[i,'startHour'] = matchData.loc[i,'start'].hour

matchData.apply(lambda x : getHourCat(x.name), axis = 1)

0      None
1      None
2      None
3      None
4      None
       ... 
96     None
97     None
98     None
99     None
100    None
Length: 101, dtype: object

In [34]:
matchData

,playerID,matchID,duration (s),start,end,championName,championId,pings,ePings,weekDay,weekDayName,startHour,startHourCat
0,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363787945,1707.0,1.68167e+12,1.68167e+12,Kaisa,145.0,11.0,6.0,NaN,NaN,NaN,Night
1,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363740926,915.0,1.68167e+12,1.68167e+12,Tristana,18.0,2.0,2.0,NaN,NaN,NaN,Night
2,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6363690986,1416.0,2023-04-16 20:11:56.902000,2023-04-16 20:35:32.783000,MissFortune,21.0,7.0,2.0,6.0,Sunday,20.0,Evening
3,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6360825821,2146.0,2023-04-14 22:27:25.403000,2023-04-14 23:03:11.648000,Kaisa,145.0,5.0,3.0,4.0,Friday,22.0,Night
4,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6359215935,1627.0,2023-04-13 20:19:14.958000,2023-04-13 20:46:22.830000,MissFortune,21.0,2.0,1.0,3.0,Thursday,20.0,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6301057974,2622.0,2023-03-03 23:18:03.961000,2023-03-04 00:01:46.195000,MissFortune,21.0,5.0,3.0,4.0,Friday,23.0,Night
97,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300979268,2050.0,2023-03-03 22:34:45.223000,2023-03-03 23:08:55.554000,MissFortune,21.0,1.0,1.0,4.0,Friday,22.0,Night
98,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300902911,1992.0,2023-03-03 21:49:52.215000,2023-03-03 22:23:05.058000,MissFortune,21.0,5.0,3.0,4.0,Friday,21.0,Night
99,p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIh...,EUW1_6300315307,1220.0,2023-03-03 14:44:01.073000,2023-03-03 15:04:21.223000,MissFortune,21.0,1.0,0.0,4.0,Friday,14.0,Night


In [87]:
matchDataFinal = pd.DataFrame({'mondayTime' : [], 'tuesdayTime' : [], 'wednesdayTime' : [], 'thursdayTime' : [], 'fridayTime' : [], 'saturdayTime' : [], 'sundayTime' : [], 'morningTime' : [], 'afternoonTime' : [], 'eveningTime' : [], 'nightTime' : [], 'mondayCount' : [], 'tuesdayCount' : [], 'wednesdayCount' : [], 'thursdayCount' : [], 'fridayCount' : [], 'saturdayCount' : [], 'sundayCount' : [], 'morningCount' : [], 'afternoonCount' : [], 'eveningCount' : [], 'nightCount' : []})
matchDataFinal

,mondayTime,tuesdayTime,wednesdayTime,thursdayTime,fridayTime,saturdayTime,sundayTime,morningTime,afternoonTime,eveningTime,...,tuesdayCount,wednesdayCount,thursdayCount,fridayCount,saturdayCount,sundayCount,morningCount,afternoonCount,eveningCount,nightCount


In [116]:
matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1) #/ matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().sum()

weekDay,0.0,1.0,2.0,3.0,4.0,5.0,6.0
playerID,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,9,12,10,19,12,18,19


In [117]:
matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1) # / matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().sum()

weekDay,0.0,1.0,2.0,3.0,4.0,5.0,6.0
playerID,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,15420.0,17745.0,15944.0,26807.0,20484.0,27926.0,27429.0


In [118]:
matchData.loc[:,['matchID', 'playerID', "startHourCat"]].groupby(['playerID','startHourCat']).count().unstack(level=1).droplevel(level = 0,axis = 1)

startHourCat,Evening,Night
playerID,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,37,64


In [120]:
matchDataFinal.loc[:,"mondayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,0]
matchDataFinal.loc[:,"tuesdayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,1]
matchDataFinal.loc[:,"wednesdayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,2]
matchDataFinal.loc[:,"thursdayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,3]
matchDataFinal.loc[:,"fridayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,4]
matchDataFinal.loc[:,"saturdayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,5]
matchDataFinal.loc[:,"sundayTime"] = matchData.loc[:,['duration (s)', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).sum().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,6]

matchDataFinal.loc[:,"mondayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,0]
matchDataFinal.loc[:,"tuesdayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,1]
matchDataFinal.loc[:,"wednesdayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,2]
matchDataFinal.loc[:,"thursdayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,3]
matchDataFinal.loc[:,"fridayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,4]
matchDataFinal.loc[:,"saturdayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,5]
matchDataFinal.loc[:,"sundayCount"] = matchData.loc[:,['matchID', 'playerID', "weekDay"]].groupby(['playerID','weekDay']).count().unstack(level=1).droplevel(level = 0,axis = 1).loc[:,6]

In [121]:
matchDataFinal

,mondayTime,tuesdayTime,wednesdayTime,thursdayTime,fridayTime,saturdayTime,sundayTime,morningTime,afternoonTime,eveningTime,...,tuesdayCount,wednesdayCount,thursdayCount,fridayCount,saturdayCount,sundayCount,morningCount,afternoonCount,eveningCount,nightCount
playerID,,,,,,,,,,,,,,,,,,,,,
p8bl0MvZHFp_3Xv0n-ct7HIqGxGD1eNKWbTEWeRkygFnIhPjg_7KsMItrmERLTuhXjh5tsLUwwX55w,15420.0,17745.0,15944.0,26807.0,20484.0,27926.0,27429.0,NaN,NaN,NaN,...,12,10,19,12,18,19,NaN,NaN,NaN,NaN
